In [27]:
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix


from typing import Dict, Union, List


import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import os

In [2]:
device = utils.get_device()

In [3]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model = HookedTransformer.from_pretrained(
    model_name,
    device=device,
    torch_dtype=torch.bfloat16,
)

model.eval()

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 49.86it/s]


Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_att

### Getting activations

In [4]:
happines_path = "/workspace/MATS-research/data/emotion_user_prompts/happiness.txt"
sadness_path = "/workspace/MATS-research/data/emotion_user_prompts/sadness.txt"

with open(happines_path, "r") as f:
    happiness_prompts = f.readlines()

with open(sadness_path, "r") as f:
    sadness_prompts = f.readlines()

#remove \n from each line
happiness_prompts = [prompt.strip() for prompt in happiness_prompts]
sadness_prompts = [prompt.strip() for prompt in sadness_prompts]

In [5]:
happiness_prompts = happiness_prompts[:500]
sadness_prompts = sadness_prompts[:500]

#do an 80/20 test train split using sklearn
from sklearn.model_selection import train_test_split

happiness_prompts_train, happiness_prompts_test = train_test_split(
    happiness_prompts, test_size=0.2, random_state=42, shuffle=True
)

sadness_prompts_train, sadness_prompts_test = train_test_split(
    sadness_prompts, test_size=0.2, random_state=42, shuffle=True
)


In [6]:
happiness_conversations = [[{"role": "user", "content": prompt}] for prompt in happiness_prompts_train]
sadness_conversations = [[{"role": "user", "content": prompt}] for prompt in sadness_prompts_train]

happiness_conversations_test = [[{"role": "user", "content": prompt}] for prompt in happiness_prompts_test]
sadness_conversations_test = [[{"role": "user", "content": prompt}] for prompt in sadness_prompts_test]

In [19]:
#Convert to tokens. Make sure to apply left padding. No generation tag.
model.tokenizer.padding_side = 'left'

happiness_tokens = model.tokenizer.apply_chat_template(
    happiness_conversations,
    add_generation_prompt=False,
    padding=True,
    return_tensors="pt"
)

sadness_tokens = model.tokenizer.apply_chat_template(
    sadness_conversations,
    add_generation_prompt=False,
    padding=True,
    return_tensors="pt"
)

happiness_tokens_test = model.tokenizer.apply_chat_template(
    happiness_conversations_test,
    add_generation_prompt=False,
    padding=True,
    return_tensors="pt"
)

sadness_tokens_test = model.tokenizer.apply_chat_template(
    sadness_conversations_test,
    add_generation_prompt=False,
    padding=True,
    return_tensors="pt"
)

In [8]:
happiness_tokens.shape

torch.Size([400, 29])

In [14]:
#check which device the tokens are on
print(happiness_tokens.device)


cpu


In [16]:
from functools import partial

def final_token_resid_hook(activation, hook, cache_dict):
    # activation is on the GPU here
    final_token_activation = activation[:, -1, :]

    cpu_activation = final_token_activation.clone().detach().cpu()

    if hook.name in cache_dict:
        # Now the concatenation happens with CPU tensors
        cache_dict[hook.name] = torch.cat([cache_dict[hook.name], cpu_activation], dim=0)
    else:
        cache_dict[hook.name] = cpu_activation

def process_tokens_in_batches(tokens, cache_dict, batch_size=8):
    """Process tokens through model in batches to avoid GPU memory issues"""
    
    # Create the hook function for this cache
    batch_final_token_resid_hook = partial(final_token_resid_hook, cache_dict=cache_dict)
    
    # Process in batches
    for i in tqdm.tqdm(range(0, tokens.shape[0], batch_size)):
        batch_tokens = tokens[i:i+batch_size].to(device)
        
        # Run model with hooks on this batch
        with model.hooks(fwd_hooks=[ (lambda name: name.endswith("hook_resid_pre"), batch_final_token_resid_hook) ] ):
            _ = model(batch_tokens)
        
        # Move batch_tokens back to CPU and delete to free GPU memory
        del batch_tokens
        
        # Clear GPU cache to prevent memory buildup
        torch.cuda.empty_cache()

In [ ]:
# Process happiness tokens in batches
happiness_cache_dict = {}
print("Processing happiness tokens through model...")
process_tokens_in_batches(happiness_tokens, happiness_cache_dict, batch_size=4)


In [18]:
# Process happiness tokens in batches
sadness_cache_dict = {}
process_tokens_in_batches(sadness_tokens, sadness_cache_dict, batch_size=4)


100%|██████████| 100/100 [00:23<00:00,  4.26it/s]


In [23]:
sadness_test_cache_dict = {}
process_tokens_in_batches(sadness_tokens_test, sadness_test_cache_dict, batch_size=4)

happiness_test_cache_dict = {}
process_tokens_in_batches(happiness_tokens_test, happiness_test_cache_dict, batch_size=4)

100%|██████████| 25/25 [00:05<00:00,  4.32it/s]


In [24]:
#clear cache
torch.cuda.empty_cache()


### Training probe

In [34]:
# easily configurable layer selection
LAYERS_TO_TRAIN = [20,25,30] 

# Directory to save the trained probe weights
PROBE_SAVE_DIR = "/workspace/MATS-research/probes"
os.makedirs(PROBE_SAVE_DIR, exist_ok=True)

# Hyperparameters (as you confirmed)
PROBE_HYPERPARAMS = {
    "epochs": 100,
    "lr": 1e-3,
    "weight_decay": 0.01, # This is the L2 regularization
    "batch_size": 32,
}

# The dimensionality of the residual stream from your model's config
d_model = model.cfg.d_model

In [33]:
class LinearProbe(nn.Module):
    """A simple linear probe that maps activations to a single logit."""
    def __init__(self, d_model: int):
        super().__init__()
        # As confirmed, a simple linear layer with no bias.
        # It maps the d_model-dimensional activation to a 1D logit.
        self.probe = nn.Linear(d_model, 1, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Input shape: [batch, d_model], Output shape: [batch]"""
        return self.probe(x).squeeze(-1)

In [40]:
def prepare_data(happy_activations, sad_activations, batch_size):
    """
    Combines happy and sad activations, creates labels, and returns a DataLoader.
    Label 1 for 'happiness', 0 for 'sadness'.
    """
    # Combine the activations into a single tensor
    all_activations = torch.cat([happy_activations, sad_activations], dim=0)

    # Create corresponding labels
    happy_labels = torch.ones(happy_activations.shape[0])
    sad_labels = torch.zeros(sad_activations.shape[0])
    all_labels = torch.cat([happy_labels, sad_labels], dim=0)

    # Create a TensorDataset and DataLoader
    dataset = TensorDataset(all_activations, all_labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# --- Training and Evaluation Functions ---

def train_probe(probe, dataloader, epochs, lr, weight_decay, batch_size, device):
    """Trains a single linear probe."""
    probe.to(device)
    probe.train()

    # The paper uses L2 regularization, which is equivalent to `weight_decay` in AdamW
    optimizer = torch.optim.AdamW(probe.parameters(), lr=lr, weight_decay=weight_decay)
    
    # BCEWithLogitsLoss is the standard for binary classification and is numerically stable
    loss_fn = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        for activations, labels in dataloader:
            activations = activations.to(device).to(torch.float32)
            labels = labels.to(device)


            logits = probe(activations)
            loss = loss_fn(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    return probe

def evaluate_probe(probe, dataloader, device):
    """Evaluates the probe's accuracy on a given dataset."""
    probe.to(device)
    probe.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for activations, labels in dataloader:
            activations = activations.to(device).to(torch.float32)
            labels = labels.to(device)

            
            logits = probe(activations)
            # A logit > 0 corresponds to a predicted probability > 0.5
            predictions = (logits > 0).long()
            
            correct += (predictions == labels).sum().item()
            total += labels.shape[0]
            
    accuracy = correct / total
    return accuracy

In [41]:
# Dictionaries to store trained probes and test accuracies
trained_probes = {}
test_accuracies = {}

print("Starting probe training and evaluation...")
print("-" * 50)

for layer in LAYERS_TO_TRAIN:
    hook_name = f"blocks.{layer}.hook_resid_pre"
    print(f"Processing Layer {layer} (Hook: {hook_name})")

    # 1. Prepare Data for the current layer
    # Training data
    happy_train_acts = happiness_cache_dict[hook_name]
    sad_train_acts = sadness_cache_dict[hook_name]
    train_loader = prepare_data(happy_train_acts, sad_train_acts, PROBE_HYPERPARAMS['batch_size'])

    # Testing data
    happy_test_acts = happiness_test_cache_dict[hook_name]
    sad_test_acts = sadness_test_cache_dict[hook_name]
    test_loader = prepare_data(happy_test_acts, sad_test_acts, PROBE_HYPERPARAMS['batch_size'])
    
    # 2. Initialize and Train the Probe
    probe = LinearProbe(d_model)
    probe = train_probe(
        probe,
        train_loader,
        device=device,
        **PROBE_HYPERPARAMS
    )
    
    # 3. Evaluate the Probe
    accuracy = evaluate_probe(probe, test_loader, device=device)
    
    # 4. Store Results and Save Weights
    trained_probes[layer] = probe
    test_accuracies[layer] = accuracy
    
    probe_save_path = os.path.join(PROBE_SAVE_DIR, f"probe_layer_{layer}.pt")
    torch.save(probe.state_dict(), probe_save_path)
    
    print(f"Layer {layer}: Test Accuracy = {accuracy:.4f}")
    print(f"Probe weights saved to: {probe_save_path}\n")

print("-" * 50)
print("All probes trained and evaluated.")

Starting probe training and evaluation...
--------------------------------------------------
Processing Layer 20 (Hook: blocks.20.hook_resid_pre)


Layer 20: Test Accuracy = 0.9300
Probe weights saved to: /workspace/MATS-research/probes/probe_layer_20.pt

Processing Layer 25 (Hook: blocks.25.hook_resid_pre)
Layer 25: Test Accuracy = 0.9550
Probe weights saved to: /workspace/MATS-research/probes/probe_layer_25.pt

Processing Layer 30 (Hook: blocks.30.hook_resid_pre)
Layer 30: Test Accuracy = 0.9400
Probe weights saved to: /workspace/MATS-research/probes/probe_layer_30.pt

--------------------------------------------------
All probes trained and evaluated.


In [ ]:

# # --- Example of Loading a Probe ---
# print("\n--- Example of loading a saved probe ---")
# layer_to_load = 15
# if layer_to_load in LAYERS_TO_TRAIN:
#     # 1. Create a new probe instance
#     loaded_probe = LinearProbe(d_model)

#     # 2. Load the saved state dictionary
#     saved_weights_path = os.path.join(PROBE_SAVE_DIR, f"probe_layer_{layer_to_load}.pt")
#     loaded_probe.load_state_dict(torch.load(saved_weights_path))

#     # 3. You can now use this probe for evaluation or inference
#     loaded_probe.to(device)
#     # Re-create the test loader to verify
#     happy_test_acts = happiness_test_cache_dict[f"blocks.{layer_to_load}.hook_resid_pre"]
#     sad_test_acts = sadness_test_cache_dict[f"blocks.{layer_to_load}.hook_resid_pre"]
#     test_loader_for_verify = prepare_data(happy_test_acts, sad_test_acts, PROBE_HYPERPARAMS['batch_size'])

#     loaded_probe_accuracy = evaluate_probe(loaded_probe, test_loader_for_verify, device=device)

#     print(f"Successfully loaded probe for layer {layer_to_load}.")
#     print(f"Original test accuracy: {test_accuracies[layer_to_load]:.4f}")
#     print(f"Loaded probe accuracy:  {loaded_probe_accuracy:.4f}")
# else:
#     print(f"Layer {layer_to_load} was not in the list of layers to train.")

### Steering with probe

In [ ]:
trained_probes

{20: LinearProbe(
   (probe): Linear(in_features=4096, out_features=1, bias=False)
 ),
 25: LinearProbe(
   (probe): Linear(in_features=4096, out_features=1, bias=False)
 ),
 30: LinearProbe(
   (probe): Linear(in_features=4096, out_features=1, bias=False)
 )}

In [46]:
neutral_prompts = [
    "What is the capital of France?",
    "Can you explain how photosynthesis works?",
    "Give me a summary of the book '1984'.",
    "What are the main differences between Python and Java?",
    "How do I bake a chocolate cake?",
    "What is the weather typically like in Tokyo in April?",
    "Can you help me practice basic Spanish phrases?",
    "Explain the concept of supply and demand.",
    "What are some common interview questions for software engineers?",
    "Can you create a simple workout plan for beginners?"
]

#convert to convsersations
neutral_conversations = [[{"role": "user", "content": prompt}] for prompt in neutral_prompts]

In [60]:
def user_teering_hook(resid_pre, hook, steering_vector, coeff):

    # if resid_pre.shape[1] == 1:
    #     return resid_pre
    
    batch_size, seq_len, d_model = resid_pre.shape
    steering_seq_len = steering_vector.shape[0]
    print('steering taking place!!!')
    resid_pre[:, -1, :] += coeff * steering_vector

In [62]:
layer = 25
coeff = 100
sampling_kwargs = dict(temperature=1.0, top_p=0.3, freq_penalty=1.0)
steering_vector = trained_probes[layer].probe.weight.data.squeeze()

# set norm of steering vector to 1
steering_vector = steering_vector / torch.norm(steering_vector)

model.tokenizer.padding_side = 'left'

# Tokenize conversations
inputs = model.tokenizer.apply_chat_template(
    neutral_conversations, 
    add_generation_prompt=False, 
    padding=True, 
    return_tensors="pt"
).to(device)

hook_fn = partial(user_teering_hook, steering_vector=steering_vector, coeff=coeff)

steering_hooks = [(f"blocks.{layer}.hook_resid_pre", hook_fn)]

# Generate with steering
with model.hooks(fwd_hooks=steering_hooks):
    outputs = model.generate(
        inputs, 
        max_new_tokens=150, 
        do_sample=True,
        **sampling_kwargs
    )

def format_output(out):
    formatted_outputs = model.to_string(out)
    print(("\n\n" + "-" * 80 + "\n\n").join(formatted_outputs))

format_output(outputs)



  1%|          | 1/150 [00:00<00:17,  8.74it/s]

steering taking place!!!
steering taking place!!!


  3%|▎         | 5/150 [00:00<00:13, 10.89it/s]

steering taking place!!!
steering taking place!!!
steering taking place!!!


  5%|▍         | 7/150 [00:00<00:12, 11.75it/s]

...

steering taking place!!!
steering taking place!!!
steering taking place!!!


 99%|█████████▉| 149/150 [00:10<00:00, 15.19it/s]

steering taking place!!!
steering taking place!!!
steering taking place!!!
steering taking place!!!


100%|██████████| 150/150 [00:10<00:00, 14.64it/s]

steering taking place!!!
<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

What is the capital of France?<|eot_id|> URLclassifierْس110alia-minus user dollars palms easusheriginal市 bri teb Kö Bralia Kag Musk兴aurcuselizeřevlesiaRecommendiece�982 adviser_bonusazz haven Mission001 Mission Bris spo测.classatik hurryalia classical Uploadedalian Guaroyer br Suitableream attitude Kansas Brad001alia latestachtenativeorre Difficulty984 instructional_GR electric 武 неп appreciation HDographer characterization Wardwand gen celebrateecalaclass irrig Eleáš SGール UploadBraduiten Boundaji erhaltenromo celebration carriage��982 servantosalrms헌 skimOSalley missionáš Eug któ�hores uncomp assignable tebolic Panel US Flor Sunshineatik.numpyaclasstempl발ailer ele spared발789 joy Köwaldreamoon MF Snackbar_spritesoyer.CommandTextosal sun brakes.volley(convertamat AI Lois.cent Wor ele recommendation-togglerinfossetattr

-----------------------------------------------

In [ ]:
#to-do Use data from empathic machines and try again.